In [ ]:
import pandas as pd
from os.path import dirname, isfile, join

# read simulation scenarios
mdir = r"../../3_models/sfincs"
rdir = r"../../4_results"
scens = pd.read_csv(join(rdir, 'sim_SCEN_indep.csv'), index_col=0).rename(columns={'h_tsw_rp': 'h_rp'})
scens

In [ ]:
# check model simulation status
import subprocess
import os
import glob
from run_sfincs import postprocess
from datetime import datetime, timedelta

def check_finished(root):
    finished = False
    failed = False
    v295 = False
    fn_log = join(root, 'sfincs.log')
    assert isfile(join(root, 'sfincs.inp'))
    if isfile(fn_log):
        mtime = os.path.getmtime(fn_log)
        mtime = datetime.fromtimestamp(mtime)
        mtime_str = mtime.strftime("%H:%M:%S")
        dt = datetime.now() - mtime
        with open(fn_log, 'r') as f:
            lines = f.readlines()
            if len(lines) == 0:
                print(f'RUNNING  UNIX: {root} {mtime_str}')
                return finished
            for l in lines:
                # if 'Rev: 295' in l:
                #     v295 = True
                if 'Simulation stopped' in l:
                    print(f'FAILED: {root}')
                    failed = True
                    break
                elif 'Simulation is finished' in l:
                    finished = True
                    break
        if not finished and dt > timedelta(seconds=3600):
            print(f'OVERDUE: {root}')
            failed = True
        if failed:
            os.rename(fn_log, join(root, 'sfincs.log.failed'))
        elif finished and isfile(join(root, 'zsmax.dat')):
            print(f'postprocessing {root}')
            postprocess(root)
        elif not finished:
            print(f'RUNNING WINDOWS: {root} {mtime_str}')
    if finished and v295:
        os.rename(fn_log, join(root, 'sfincs.log.org'))
        if isfile(join(root, 'gis', 'hmax.tif')):
            os.rename(join(root, 'gis', 'hmax.tif'), join(root, 'gis', 'hmax_org.tif'))
        if isfile(join(root, 'fig', 'hmax.png')):
            os.rename(join(root, 'fig', 'hmax.png'), join(root, 'fig', 'hmax_org.png'))
        finished = False
    return finished

# runs = [dirname(fn) for fn in glob.glob(join(mdir, '*', 'sfincs.inp')) if not check_finished(dirname(fn))]
runs = [join(mdir, name) for name in scens['scen'] if not check_finished(join(mdir, name))]
n = len(runs)
print(n)